In [2]:
import os
import sys
import urllib, io

import numpy as np
import scipy.stats as stats
import pandas as pd

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

### setup paths

In [3]:
## directory & file hierarchy
proj_dir = os.path.abspath('..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'analysis') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'python'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       
    
## Assign variables within imported analysis helpers
import analysis_helpers as h
if sys.version_info[0]>=3:
    from importlib import reload
reload(h)

<module 'analysis_helpers' from '/Users/judithfan/block_construction/analysis/analysis_helpers.py'>

### connect to db [skip this if on compute server that is behind vpn]

#### remember to establish tunnel to cogtoolslab server first: `ssh -fNL 27017:127.0.0.1:27017 USER@cogtoolslab.org`

In [6]:
## set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'cogtoolslab.org' ## cocolab ip address

## have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['block_construction']
coll = db['silhouette']

## which iteration name should we use?
iterationName = 'testing'

### query over mongodb records and organize into tidy dataframe

In [10]:
coll.find_one()

{'_id': ObjectId('5da60d7d34ba85035929396e'),
 'iterationName': 'testing',
 'dataType': 'block',
 'gameID': 'GAMEID_PLACEHOLDER',
 'time': 3683.549999987008,
 'time_absolute': 1571163517322.0}

In [53]:
## define how we will initially query records in the database
query = coll.find({'blockVertices':{'$exists':True},'iterationName':iterationName })

In [54]:
## convert into pandas dataframe
df = pd.DataFrame(list(query.sort('time_absolute')))

In [57]:
coll.distinct('iterationName')

['testing']

In [58]:
df

,_id,blockBodyTest,blockCenterX,blockCenterY,blockHeight,blockVertices,blockWidth,dataType,gameID,iterationName,time,timeAbsolute
0,5da6158f34ba85035929397e,"{'id': 5, 'type': 'body', 'label': 'Rectangle ...",625,882,80,"[{'x': 545, 'y': 842}, {'x': 705, 'y': 842}, {...",160,block,GAMEID_PLACEHOLDER,testing,1648.455,1.571166e+12
1,5da6159e34ba85035929397f,"{'id': 5, 'type': 'body', 'label': 'Rectangle ...",619,878,80,"[{'x': 539, 'y': 838}, {'x': 699, 'y': 838}, {...",160,block,GAMEID_PLACEHOLDER,testing,1625.005,1.571166e+12
2,5da615a434ba850359293980,"{'id': 5, 'type': 'body', 'label': 'Rectangle ...",609,806,240,"[{'x': 569, 'y': 686}, {'x': 649, 'y': 686}, {...",80,block,GAMEID_PLACEHOLDER,testing,1712.595,1.571166e+12
3,5da615e334ba850359293981,"{'id': 5, 'type': 'body', 'label': 'Rectangle ...",625,878,80,"[{'x': 545, 'y': 838}, {'x': 705, 'y': 838}, {...",160,block,GAMEID_PLACEHOLDER,testing,1356.785,1.571166e+12
4,5da615e934ba850359293982,"{'id': 5, 'type': 'body', 'label': 'Rectangle ...",587,872,80,"[{'x': 507, 'y': 832}, {'x': 667, 'y': 832}, {...",160,block,GAMEID_PLACEHOLDER,testing,2076.630,1.571166e+12
5,5da615ee34ba850359293983,"{'id': 5, 'type': 'body', 'label': 'Rectangle ...",621,874,80,"[{'x': 541, 'y': 834}, {'x': 701, 'y': 834}, {...",160,block,GAMEID_PLACEHOLDER,testing,1244.405,1.571166e+12
6,5da615fb34ba850359293984,"{'id': 5, 'type': 'body', 'label': 'Rectangle ...",599,858,80,"[{'x': 519, 'y': 818}, {'x': 679, 'y': 818}, {...",160,block,GAMEID_PLACEHOLDER,testing,1976.255,1.571166e+12
7,5da6163934ba850359293985,"{'id': 5, 'type': 'body', 'label': 'Rectangle ...",607,882,80,"[{'x': 527, 'y': 842}, {'x': 687, 'y': 842}, {...",160,block,GAMEID_PLACEHOLDER,testing,1964.320,1.571166e+12
8,5da6164b34ba850359293986,"{'id': 6, 'type': 'body', 'label': 'Rectangle ...",613,810,80,"[{'x': 493, 'y': 770}, {'x': 733, 'y': 770}, {...",240,block,GAMEID_PLACEHOLDER,testing,19913.665,1.571166e+12
9,5da6164d34ba850359293987,"{'id': 7, 'type': 'body', 'label': 'Rectangle ...",621,714,80,"[{'x': 501, 'y': 674}, {'x': 741, 'y': 674}, {...",240,block,GAMEID_PLACEHOLDER,testing,22240.650,1.571166e+12


#### TODOs

1. insert records from working demo that have updated iterationName: 'testing2' ?
2. insert trialNum into block_data object
3. create explore_data object that saves data from exploration phase of each trial
4. create build_data object that saves data from build phase of each trial
5. take blocks from single phase of single trial and display them here in this notebook